<a href="https://colab.research.google.com/github/emanuelmejia/courses/blob/main/Emanuel_Mejia_Week_09_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading & Processing

In [ ]:
import pandas as pd
import altair as alt
from vega_datasets import data

In [ ]:
alt.__version__

'4.2.2'

In [ ]:
# This book requires the json file found at:
# https://public.opendatasoft.com/explore/dataset/georef-united-states-of-america-county/export/?disjunctive.ste_code&disjunctive.ste_name&disjunctive.coty_code&disjunctive.coty_name&sort=year
# Named as 'counties.json'

from google.colab import files
files.upload()

Saving counties.json to counties.json


In [ ]:
county_df = pd.read_json("/content/counties.json")
county_df['name'] = pd.DataFrame(county_df.coty_name.tolist(), index= county_df.index)
county_df['id'] = pd.DataFrame(county_df.coty_code.tolist(), index= county_df.index)
county_df['state'] = pd.DataFrame(county_df.ste_name.tolist(), index= county_df.index)

county_df['id'] = county_df['id'].astype(int)

county_df = county_df.drop(county_df.loc[:, 'geo_point_2d':'coty_gnis_code'].columns, axis=1)
county_df.head()

In [ ]:
counties = alt.topo_feature(data.us_10m.url, feature = 'counties')
source = data.unemployment()

full_df = pd.merge(source, county_df, how = 'left', on = 'id')
full_df.fillna('Not_Found', inplace = True)

# Graphing

In [ ]:
chart = alt.Chart(counties).mark_geoshape(
            tooltip = True,
        ).encode(
            color = alt.Color('rate:Q',
                              title = 'Rate',
                              scale = alt.Scale(
                                  scheme = 'purples',
                                  domain = [0, 0.15]
                              ),
                              legend = alt.Legend(format = '0.1%')),
            tooltip = [
                alt.Tooltip('id:N', title = 'FIPS Code'),
                alt.Tooltip('name:N', title = 'County'),
                alt.Tooltip('state:N', title = 'State'),
                alt.Tooltip('rate:Q', format='0.1%', title = 'Rate')
            ]
        ).transform_lookup(
            lookup = 'id',
            from_=alt.LookupData(full_df, 'id', list(full_df.columns))
        ).project(
            type = 'albersUsa'
        ).properties(
            title={
              "anchor": "start",
              "text": "USA County Unemployment",
              "color": "slateblue",
              "font": "Sans-Serif",
              "fontSize": 30,
              "fontWeight": "bold",
              "subtitle": "As a percentage of the total labor force",
              "subtitleColor": "dimgray",
              "subtitleFont" : "Sans-Serif",
              "subtitleFontSize": 15,
              "subtitleFontStyle": "italic"
            },
            width = 600,
            height = 400
        ).configure_legend(
            titleColor='rebeccapurple',
            titleFontSize = 12,
            labelColor = 'dimgray',
            labelFontSize = 10,
            labelFontWeight = 'bold',
            strokeColor='indigo',
            fillColor='ghostwhite',
            padding=10,
            cornerRadius=10,
        )

chart

alt.Chart(...)